In [1]:
from gurobipy import*
import numpy as np
#i is the number of months
i=range(12)

#j is the production mode of normal production and 
j=range(2)

# k is the month sold
k=range(12)

#name of the months used for printing the results
names=['January','Feburary','March','April','May','June','July','August','September','October','November','December']

#name of the production mode used when printing results
modes=['Regular mode','Overtime']

#demand in each month
d_k=[800,900,1100,400,700,1100,700,900,800,700,800,1100]

#capacity constraint for each production mode
u_j=[800,200]

#cost of production
c_j=[10,12]

#storage cost per month
h=1

#perishable after 2 months
perishable=2

m=Model("Project2_question1")

x=m.addVars(i,j,k,vtype=GRB.INTEGER,name='Amount_produced_in_i_month_of_type_j_sold_in_month_k')


#assigning zero to values out of scope and d,f,g are used for iteration
for d in i:
    for f in j:
        for g in k:
            if g<d:
                m.addConstr(x[d,f,g]==0)
            elif g>d+perishable:
                m.addConstr(x[d,f,g]==0)

#production cost + storage cost
objective=quicksum(c_j[f]*(quicksum(x[d,f,g] for g in k for d in i)) for f in j)+quicksum(h*(g-d)*x[d,f,g] for d in i for f in j for g in k)

#objective function

m.setObjective(objective, GRB.MINIMIZE)

#demand constraint
m.addConstrs((quicksum(x[d,f,g] for f in j for d in i) == d_k[g] for g in k), name="Demandconstraint")

#capasity constraint
m.addConstrs((quicksum(x[d,f,g] for g in k)<=u_j[f] for f in j for d in i),name='capacityconstriant')

#non-negative constraint
m.addConstrs((x[d,f,g]>=0 for d in i for f in j for g in k),name="non_negativeconstraint")

m.optimize()

#printing optimal objective value
print("\n\n\nOptimal total production and storage cost for the next 12 months is equal to",m.objVal ,"dollar\n\n")

#representing answer in a structured way
for d in i:
    print("In",names[d],":")
    for f in j:
        for g in k:
            if g==d:
                print("\tProduce",x[d,f,g].X, "in",modes[f],"to be sold in",names[g])
            elif g==d+1:
                print("\tProduce",x[d,f,g].X, "in",modes[f],"to be sold in",names[g])
            elif g==d+2:
                print("\tProduce",x[d,f,g].X, "in",modes[f],"to be sold in",names[g])
            else:
                continue
                
    


Academic license - for non-commercial use only
Optimize a model with 546 rows, 288 columns and 1086 nonzeros
Variable types: 0 continuous, 288 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 516 rows and 228 columns
Presolve time: 0.00s
Presolved: 30 rows, 60 columns, 118 nonzeros
Variable types: 0 continuous, 60 integer (0 binary)
Found heuristic solution: objective 104888.00000

Root relaxation: objective 1.021000e+05, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    102100.00000 102100.000  0.00%     -    0s

Explored 0 nodes (13 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 available processors)

Solution count 2: 102100 104888 

Optimal solution found (toleranc

In [4]:
from numpy import*
from gurobipy import*
import math

#defining distance
distance={}

#r and t are used for iteration
r=0
t=0

#opening dat file and assigning it to f
f=open("PC001.dat")
for line in f:
    a=line.rstrip()
    if not len(a)>0:
        continue
    else:
        a=line.split()
        if a[0]=='N:':
            points=a[1]
        elif a[0]=='P:':
            r_centers=a[1]
        else:
            continue
#assigning the values read from dat file to n, c, and p so we can use it in further program

#number of points is given as n
n=int(points)

#number of candidate points for becoming center
c=int(points)

#number of centers we need
p=int(r_centers)

#the following set of code is used for importing distance data from dat file to distance array
f=open("PC001.dat")
for line in f:
    line=line.split()
    if not len(line)>3:
        continue
    else:
        for h in line:
            distance[r,t]=h
            t=t+1
        t=0
        r=r+1       
m=Model("IEMProject")

#add variables

#x is 1 if the point i is assigned to center j; otherwise zero
x={}

#y is 1 if the point j is a center
y={}

#z is the maximum distance between the point to its center
z=m.addVar(obj=1,vtype="C",name='z')

#add objective
m.setObjective(z, GRB.MINIMIZE)

#add variables
for j in range(c):
    y[j]=m.addVar(obj=0,vtype="B",name='y')
    for i in range(n):
        x[(i,j)]=m.addVar(obj=0,vtype="B",name='x')

#add constraints
for i in range(n):
    m.addConstr(y[i]+quicksum(x[(i,j)] for j in range(c)) ==1)
m.addConstr(quicksum(y[j] for j in range(c))==p)

for j in range(c):
    for i in range(n):
        m.addConstr(x[(i,j)] <= y[j])
        
for i in range(n):
    for j in range(c):
        m.addConstr(x[(i,j)]*distance[(i,j)] <= z)
        
        
m.optimize()
m.write("IEMProject.sol")
m.write("IEMProject.lp")

#printing soloution
if m.status == GRB.OPTIMAL:
        print("\n\n\nThe maximum distance between the non-center to its center is", m.objVal, "\n\n")
        for j in range(c):
            if y[j].X==1:
                print("The point", j+1, "is a center")
                for i in range(n):
                    if x[(i,j)].X==1:
                        print("\tThe non-center point", i+1, "is assigned to center point", j+1,", and the distance is",distance[(i,j)])
                    else:
                        continue
            else:
                continue
else:
    print('Optimum not found. Gurobi status code: ',m.status)
        


Optimize a model with 211 rows, 111 columns and 510 nonzeros
Variable types: 1 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
Found heuristic solution: objective 906.0000000
Presolve removed 26 rows and 13 columns
Presolve time: 0.00s
Presolved: 185 rows, 98 columns, 455 nonzeros
Variable types: 0 continuous, 98 integer (97 binary)

Root relaxation: objective 2.308423e+01, 107 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   23.08423    0   87  906.00000   23.08423  97.5%     -    0s
H    0     0                     805.0000000   23.08423  97.1%     -    0s
H    0     0                     700.0000000   23.08423  96.7%     -    0s
H    0     0                     467.0000000   23.08423  95.1%     -

In [6]:
from numpy import*
from gurobipy import*
import math

#defining distance
distance={}

#r and t are used for iteration
r=0
t=0

#opening dat file and assigning it to f
f=open("PC002.dat")
for line in f:
    a=line.rstrip()
    if not len(a)>0:
        continue
    else:
        a=line.split()
        if a[0]=='N:':
            points=a[1]
        elif a[0]=='P:':
            r_centers=a[1]
        else:
            continue
#assigning the values read from dat file to n, c, and p so we can use it in further program

#number of points is given as n
n=int(points)

#number of candidate points for becoming center
c=int(points)

#number of centers we need
p=int(r_centers)

#the following set of code is used for importing distance data from dat file to distance array
f=open("PC002.dat")
for line in f:
    line=line.split()
    if not len(line)>3:
        continue
    else:
        for h in line:
            distance[r,t]=h
            t=t+1
        t=0
        r=r+1       
m=Model("IEMProject")

#add variables

#x is 1 if the point i is assigned to center j; otherwise zero
x={}

#y is 1 if the point j is a center
y={}

#z is the maximum distance between the point to its center
z=m.addVar(obj=1,vtype="C",name='z')

#add objective
m.setObjective(z, GRB.MINIMIZE)

#add variables
for j in range(c):
    y[j]=m.addVar(obj=0,vtype="B",name='y')
    for i in range(n):
        x[(i,j)]=m.addVar(obj=0,vtype="B",name='x')

#add constraints
for i in range(n):
    m.addConstr(y[i]+quicksum(x[(i,j)] for j in range(c)) ==1)
m.addConstr(quicksum(y[j] for j in range(c))==p)

for j in range(c):
    for i in range(n):
        m.addConstr(x[(i,j)] <= y[j])
        
for i in range(n):
    for j in range(c):
        m.addConstr(x[(i,j)]*distance[(i,j)] <= z)
        
        
m.optimize()
m.write("IEMProject.sol")
m.write("IEMProject.lp")

#printing soloution
if m.status == GRB.OPTIMAL:
        print("\n\n\nThe maximum distance between the non-center to its center is", m.objVal, "\n\n")
        for j in range(c):
            if y[j].X==1:
                print("The point", j+1, "is a center")
                for i in range(n):
                    if x[(i,j)].X==1:
                        print("\tThe non-center point", i+1, "is assigned to center point", j+1,", and the distance is",distance[(i,j)])
                    else:
                        continue
            else:
                continue
else:
    print('Optimum not found. Gurobi status code: ',m.status)
        



Optimize a model with 20101 rows, 10101 columns and 50100 nonzeros
Variable types: 1 continuous, 10100 integer (10100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 9919.0000000
Presolve removed 109 rows and 8 columns
Presolve time: 0.18s
Presolved: 19992 rows, 10093 columns, 49974 nonzeros
Variable types: 0 continuous, 10093 integer (10092 binary)

Root relaxation: objective 1.307101e+01, 11427 iterations, 0.88 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   17.22253    0 9179 9919.00000   17.22253   100%     -    6s
H    0     0                    2231.0000000   17.22253  99.2%     -    6s
     0     0   28.28746    0 2001 2231.00000   28.28746  98.7%     -   10s
     0     0   45.19025    0 1905 2231.000

	The non-center point 81 is assigned to center point 33 , and the distance is 467
	The non-center point 88 is assigned to center point 33 , and the distance is 408
	The non-center point 94 is assigned to center point 33 , and the distance is 306
The point 41 is a center
	The non-center point 3 is assigned to center point 41 , and the distance is 516
	The non-center point 11 is assigned to center point 41 , and the distance is 566
	The non-center point 12 is assigned to center point 41 , and the distance is 477
	The non-center point 45 is assigned to center point 41 , and the distance is 505
	The non-center point 51 is assigned to center point 41 , and the distance is 648
	The non-center point 52 is assigned to center point 41 , and the distance is 606
	The non-center point 60 is assigned to center point 41 , and the distance is 253
	The non-center point 63 is assigned to center point 41 , and the distance is 129
The point 62 is a center
	The non-center point 9 is assigned to center poi

In [7]:
from numpy import*
from gurobipy import*
import math

#defining distance
distance={}

#r and t are used for iteration
r=0
t=0

#opening dat file and assigning it to f
f=open("PC003.dat")
for line in f:
    a=line.rstrip()
    if not len(a)>0:
        continue
    else:
        a=line.split()
        if a[0]=='N:':
            points=a[1]
        elif a[0]=='P:':
            r_centers=a[1]
        else:
            continue
#assigning the values read from dat file to n, c, and p so we can use it in further program

#number of points is given as n
n=int(points)

#number of candidate points for becoming center
c=int(points)

#number of centers we need
p=int(r_centers)

#the following set of code is used for importing distance data from dat file to distance array
f=open("PC003.dat")
for line in f:
    line=line.split()
    if not len(line)>3:
        continue
    else:
        for h in line:
            distance[r,t]=h
            t=t+1
        t=0
        r=r+1       
m=Model("IEMProject")

#add variables

#x is 1 if the point i is assigned to center j; otherwise zero
x={}

#y is 1 if the point j is a center
y={}

#z is the maximum distance between the point to its center
z=m.addVar(obj=1,vtype="C",name='z')

#add objective
m.setObjective(z, GRB.MINIMIZE)

m.params.timelimit = 600

#add variables
for j in range(c):
    y[j]=m.addVar(obj=0,vtype="B",name='y')
    for i in range(n):
        x[(i,j)]=m.addVar(obj=0,vtype="B",name='x')

#add constraints
for i in range(n):
    m.addConstr(y[i]+quicksum(x[(i,j)] for j in range(c)) ==1)
m.addConstr(quicksum(y[j] for j in range(c))==p)

for j in range(c):
    for i in range(n):
        m.addConstr(x[(i,j)] <= y[j])
        
for i in range(n):
    for j in range(c):
        m.addConstr(x[(i,j)]*distance[(i,j)] <= z)
        
m.optimize()        
m.write("IEMProject.sol")
m.write("IEMProject.lp")

#printing soloution

if m.status == GRB.OPTIMAL:
    print("do nothing")
else:
    print("\n\n\nThe maximum distance between the non-center to its center is", m.objVal, "\n\n")
    for j in range(c):
        if y[j].X==1:
            print("The point", j+1, "is a center")
            for i in range(n):
                if x[(i,j)].X==1:
                    print("\tThe non-center point", i+1, "is assigned to center point", j+1,", and the distance is",distance[(i,j)])
                else:
                    continue
        else:
            continue


Changed value of parameter timelimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 80201 rows, 40201 columns and 200200 nonzeros
Variable types: 1 continuous, 40200 integer (40200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Found heuristic solution: objective 19982.000000
Presolve removed 209 rows and 8 columns
Presolve time: 0.79s
Presolved: 79992 rows, 40193 columns, 199974 nonzeros
Variable types: 0 continuous, 40193 integer (40192 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   40762    8.9055486e+00   3.177043e+03   0.000000e+00      6s
   43376    1.1699155e+01   1.023347e+03   0.000000e+00     11s
   45445    1.2673915e+01   7.130612e+02   0.000000e+00     15s
   48151    1.3117957e+01   1.119308e+01   0.000000e+00     20s
   48252    1.2965667e+01   0.000000e+00   0.0

  1130   730  228.00000  606 1711 1927.00000  181.00000  90.6%   799  468s
  1219   802  235.00000  653 1630 1927.00000  181.00000  90.6%   781  482s
  1280   854  240.00000  673 1644 1927.00000  181.00000  90.6%   769  488s
  1401   958  251.21235  739 1439 1927.00000  181.00000  90.6%   745  495s
  1533  1057  269.00000  815 1344 1927.00000  181.00000  90.6%   721  503s
  1663  1160  278.00000  871 1247 1927.00000  181.00000  90.6%   703  510s
H 1810  1158                    1836.0000000  181.00000  90.1%   681  524s
  1812  1157  312.37410  926 1051 1836.00000  181.00000  90.1%   682  531s
  1999  1266  413.00000 1001  851 1836.00000  188.64820  89.7%   653  539s
H 2174  1104                    1633.0000000  189.00000  88.4%   631  573s
  2175  1104 1276.00000  783 8778 1633.00000  189.00000  88.4%   631  575s
  2177  1105 1429.00000  252 1785 1633.00000  277.00000  83.0%   630  580s
  2189  1113 1426.00000 1010 2073 1633.00000  330.67788  79.8%   627  585s
  2201  1121 1219.00000 1